# Vanishing Point Detection  (_Canny + Probabilistic Hough Transform + RANSAC_)

Classical but still very effective approach for one-point perspective scenes.Example:
- roads
- railways
- corridors
- architecture

**Pipeline overview:**  
1. Grayscale + strong edge detection (_Canny_)  
2. Line segment detection (_HoughLinesP_)  
3. Robust vanishing point estimation via `RANSAC` on line intersections  

**Still very useful when you need:** 
• interpretable results  
• low compute / no deep learning  
• good initialization for `SLAM` / `calibration` / `3D reconstruction`

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from tools.tools import LearnTools

learn_tools = LearnTools()
%matplotlib inline

## 1. Load & Prepare Image

In [ ]:
# Load Image form Image URLs

# img_url = "https://images.unsplash.com/photo-1506905925346-21bda4d32df4"
img_url = "https://i.ibb.co.com/BVSYcmyY/joey-kyber-GPxgi4-J82-E4-unsplash.jpg"



if os.path.exists("testImage.jpg"):
    image = cv2.imread("testImage.jpg")
else:
    pil_image = await learn_tools.get_image(
            img_url=img_url,
            padding=0
        )
    pil_image.save("testImage.jpg", "JPEG")
    image = learn_tools.pil_to_cv2(pil_image=pil_image)

# Convert to grayscale (Equalization is typically performed on single channels)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Gray Image', 'image': gray, 'cmap': 'gray'}
        ]
    )

## 2. Edge Detection (Canny)

In [ ]:
# ── Most important parameters ───────────────────────────────────────
CANNY_LOW    = 35
CANNY_HIGH   = 180
CANNY_APERTURE = 3

edges = cv2.Canny(
        image=gray,
        threshold1 = CANNY_LOW,
        threshold2 = CANNY_HIGH,
        apertureSize = CANNY_APERTURE,
        L2gradient = True
    )


# Display Result
learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Gray Image', 'image': gray, 'cmap': 'gray'},
            {"title": "Canny Edges", "image": edges, "cmap": "gray"}
        ]
    )

## 3. Line Detection – Probabilistic Hough Transform

In [ ]:
# ── HoughLinesP parameters ──────────────────────────────────────────
HOUGH_RHO         = 1
HOUGH_THETA       = np.pi / 180
HOUGH_THRESHOLD   = 90      # min votes
MIN_LINE_LENGTH   = 100
MAX_LINE_GAP      = 15

lines = cv2.HoughLinesP(
        image=edges,
        rho= HOUGH_RHO,
        theta= HOUGH_THETA,
        threshold= HOUGH_THRESHOLD,
        minLineLength= MIN_LINE_LENGTH,
        maxLineGap= MAX_LINE_GAP
    )

print(f"Detected {len(lines) if lines is not None else 0} line segments")

# Visualization
line_img = image.copy()
if lines is not None:
    for [[x1,y1,x2,y2]] in lines:
        cv2.line(
            img=line_img,
            pt1=(x1,y1),
            pt2=(x2,y2),
            color=(0, 255, 100),
            thickness=2,
            lineType=cv2.LINE_AA
        )

# Display Result
learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Gray Image', 'image': gray, 'cmap': 'gray'},
            {"title": "Canny Edges", "image": edges, "cmap": "gray"},
            {"title": "Detected Line Segments", "image": line_img}
        ]
    )

## 4. RANSAC-based Vanishing Point Estimation

In [ ]:
def line_to_homogeneous(x1,y1,x2,y2):
    """ Line in homogeneous form: ax + by + c = 0 """
    a = y1 - y2
    b = x2 - x1
    c = x1*y2 - x2*y1
    return np.array([a, b, c], dtype=float)


def intersection(l1, l2, eps=1e-8):
    """ Intersection point of two lines in homogeneous coordinates """
    p = np.cross(l1, l2)
    if abs(p[2]) < eps:
        return None  # parallel / same line
    return p[:2] / p[2]


def point_line_distance(point, line):
    """ Distance from point to line (ax+by+c=0) """
    x, y = point
    a, b, c = line
    return abs(a*x + b*y + c) / np.sqrt(a*a + b*b + 1e-10)


# Prepare lines in normal form
if lines is None:
    print("No lines detected - cannot continue")
else:
    lines_h = [line_to_homogeneous(x1,y1,x2,y2) for [[x1,y1,x2,y2]] in lines]

    best_vp = None
    best_inliers = 0
    best_mask = None

    N_ITERATIONS   = 1200
    INLIER_TH      = 4.0       # pixels
    MIN_INLIERS    = 15

    n_lines = len(lines_h)

    for _ in range(N_ITERATIONS):
        idx = np.random.choice(n_lines, 2, replace=False)
        l1, l2 = lines_h[idx[0]], lines_h[idx[1]]

        vp_candidate = intersection(l1, l2)
        if vp_candidate is None:
            continue

        # Count inliers
        distances = [point_line_distance(vp_candidate, l) for l in lines_h]
        inliers = np.array(distances) < INLIER_TH
        count = np.sum(inliers)

        if count > best_inliers:
            best_inliers = count
            best_vp = vp_candidate
            best_mask = inliers

    print(f"Best model found with {best_inliers} inliers ({best_inliers/n_lines:.1%})")

    # ── Visualization ───────────────────────────────────────────────
    result = image.copy()

    # Draw all lines (faint)
    for i, [[x1,y1,x2,y2]] in enumerate(lines):
        color = (80,80,255) if not best_mask[i] else (0,220,50)
        cv2.line(result, (x1,y1), (x2,y2), color, 2, cv2.LINE_AA)

    if best_vp is not None:
        x, y = map(int, best_vp)
        # Draw big red vanishing point
        cv2.circle(result, (x,y), 12, (255,30,30), -1, cv2.LINE_AA)
        cv2.circle(result, (x,y), 18, (255,255,255), 3, cv2.LINE_AA)


    # Display Result
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Gray Image', 'image': gray, 'cmap': 'gray'},
            {"title": "Canny Edges", "image": edges, "cmap": "gray"},
            {"title": "Detected Line Segments", "image": line_img},
            {"title": f'Vanishing Point (RANSAC)  -  {best_inliers} inliers', "image": result}
        ]
    )

## Quick tuning guide

| Stage          | Most sensitive parameters                          | Typical range / advice                             |
|---------------|----------------------------------------------------|-----------------------------------------------------|
| Canny         | low/high threshold                                 | low: 50–120, high: 120–250                         |
| HoughP        | threshold, minLineLength, maxLineGap               | threshold 70–140, minLen 60–180, gap 8–25          |
| RANSAC        | inlier threshold (px), nb iterations               | 3–8 pixels, 800–2500 iterations                    |